# Structured outputs on Amazon Bedrock

**Build reliable AI applications with schema-compliant JSON responses**

This notebook demonstrates how to leverage structured outputs on Amazon Bedrock to get validated JSON responses from foundation models. Through practical examples, you'll discover how to eliminate parsing errors, enforce type safety, and build production-ready AI applications.

---

## What you'll learn

- 📊 **JSON Schema output format**: Control the model's response structure with schema compliance
- 🔧 **Strict tool use**: Validate tool parameters for reliable agentic workflows
- 🎯 **Combined usage**: Use both features together for end-to-end schema validation
- 💡 **Best practices**: Production-ready patterns and error handling

---

## 1. Setup and configuration

Let's start by installing dependencies and configuring our Amazon Bedrock client.

In [ ]:
# Install required packages (uncomment if needed)
# !pip install boto3 --upgrade

In [ ]:
# Required libraries
import boto3
import json
from datetime import datetime
from botocore.exceptions import ClientError

# Configuration
REGION = "us-east-1"  # Update to your preferred region
MODEL_ID = "us.anthropic.claude-opus-4-5-20251101-v1:0"

# Initialize the Bedrock Runtime client
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name=REGION
)

print("✅ Setup complete!")
print(f"📍 Region: {REGION}")
print(f"🤖 Model: {MODEL_ID}")

---

## 2. JSON Schema output format with Converse API

The Converse API provides a unified interface for conversational AI. With structured outputs, you can control the model's response format and receive schema-compliant JSON.

**Key concept:** The `outputConfig.textFormat` parameter specifies your JSON schema, and Amazon Bedrock constrains the model's output to conform to that structure.

### 2.1 Quick start: basic lead extraction

This example matches the blog post—a simple 4-field schema to extract customer information from an email.

In [ ]:
# Define your JSON schema (matches the blog example)
extraction_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "description": "Customer name"},
        "email": {"type": "string", "description": "Customer email address"},
        "plan_interest": {"type": "string", "description": "Product plan of interest"},
        "demo_requested": {"type": "boolean", "description": "Whether a demo was requested"}
    },
    "required": ["name", "email", "plan_interest", "demo_requested"],
    "additionalProperties": False  # CRITICAL: Must be False for structured outputs
}

# Make the request with structured outputs
response = bedrock_runtime.converse(
    modelId=MODEL_ID,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "text": "Extract the key information from this email: John Smith (john@example.com) is interested in our Enterprise plan and wants to schedule a demo for next Tuesday at 2pm."
                }
            ]
        }
    ],
    inferenceConfig={
        "maxTokens": 1024
    },
    outputConfig={
        "textFormat": {
            "type": "json_schema",
            "structure": {
                "jsonSchema": {
                    "schema": json.dumps(extraction_schema),
                    "name": "lead_extraction",
                    "description": "Extract lead information from customer emails"
                }
            }
        }
    }
)

# Parse the schema-compliant JSON response
result = json.loads(response["output"]["message"]["content"][0]["text"])

print("📊 Extracted Lead Information:")
print("=" * 50)
print(json.dumps(result, indent=2))

### 2.2 Extended example: more fields and enums

Building on the quick start, this example adds more fields including enum constraints for categorical values.

In [ ]:
# Extended schema with enum constraints
lead_schema_extended = {
    "type": "object",
    "properties": {
        "name": {
            "type": "string",
            "description": "Full name of the contact"
        },
        "email": {
            "type": "string",
            "description": "Email address"
        },
        "company": {
            "type": "string",
            "description": "Company or organization name"
        },
        "plan_interest": {
            "type": "string",
            "enum": ["Starter", "Professional", "Enterprise"],
            "description": "Product plan of interest"
        },
        "demo_requested": {
            "type": "boolean",
            "description": "Whether a product demo was requested"
        },
        "urgency": {
            "type": "string",
            "enum": ["low", "medium", "high"],
            "description": "Urgency level of the inquiry"
        }
    },
    "required": ["name", "email", "plan_interest", "demo_requested", "urgency"],
    "additionalProperties": False
}

# Sample email to extract information from
sample_email = """
Hi there,

I'm Sarah Chen from TechStart Inc. (sarah.chen@techstart.io). We've been evaluating 
AI solutions for our customer service team and your Enterprise plan looks perfect 
for our needs.

We're hoping to make a decision by end of month, so I'd love to schedule a demo 
as soon as possible - ideally this week if you have availability.

Looking forward to hearing from you!

Best,
Sarah
"""

# Make the structured output request
response = bedrock_runtime.converse(
    modelId=MODEL_ID,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "text": f"Extract the lead information from this email:\n\n{sample_email}"
                }
            ]
        }
    ],
    inferenceConfig={
        "maxTokens": 1024,
        "temperature": 0
    },
    outputConfig={
        "textFormat": {
            "type": "json_schema",
            "structure": {
                "jsonSchema": {
                    "schema": json.dumps(lead_schema_extended),
                    "name": "lead_extraction_extended",
                    "description": "Extract detailed lead information from customer emails"
                }
            }
        }
    }
)

# Parse the schema-compliant JSON
result = json.loads(response["output"]["message"]["content"][0]["text"])

print("📊 Extracted Lead Information (Extended):")
print("=" * 50)
print(json.dumps(result, indent=2))
print("\n✅ All required fields present with enum values constrained!")

In [ ]:
### 2.3 Complex nested structures

Structured outputs handles nested objects and arrays, enabling extraction of complex hierarchical data like invoices.

In [ ]:
# Schema for extracting invoice data with nested line items
invoice_schema = {
    "type": "object",
    "properties": {
        "invoice_number": {
            "type": "string",
            "description": "Unique invoice identifier"
        },
        "date": {
            "type": "string",
            "format": "date",
            "description": "Invoice date in YYYY-MM-DD format"
        },
        "customer": {
            "type": "object",
            "properties": {
                "name": {"type": "string"},
                "address": {"type": "string"},
                "email": {"type": "string"}
            },
            "required": ["name"],
            "additionalProperties": False
        },
        "line_items": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "description": {"type": "string"},
                    "quantity": {"type": "integer"},
                    "unit_price": {"type": "number"},
                    "total": {"type": "number"}
                },
                "required": ["description", "quantity", "unit_price", "total"],
                "additionalProperties": False
            }
        },
        "subtotal": {"type": "number"},
        "tax_rate": {"type": "number"},
        "tax_amount": {"type": "number"},
        "total_amount": {"type": "number"},
        "payment_status": {
            "type": "string",
            "enum": ["pending", "paid", "overdue", "cancelled"]
        }
    },
    "required": ["invoice_number", "date", "customer", "line_items", "total_amount", "payment_status"],
    "additionalProperties": False
}

# Sample invoice text
invoice_text = """
INVOICE #INV-2024-0892
Date: January 15, 2024

Bill To:
Acme Corporation
123 Business Ave, Suite 100
San Francisco, CA 94105
billing@acmecorp.com

Items:
1. Cloud Computing Services (Annual) - 1 x $12,000.00 = $12,000.00
2. Premium Support Package - 1 x $2,400.00 = $2,400.00  
3. Data Storage (500GB) - 12 x $50.00 = $600.00

Subtotal: $15,000.00
Tax (8.5%): $1,275.00
TOTAL DUE: $16,275.00

Status: Payment Received - Thank you!
"""

response = bedrock_runtime.converse(
    modelId=MODEL_ID,
    messages=[
        {
            "role": "user",
            "content": [{"text": f"Extract all invoice details from this document:\n\n{invoice_text}"}]
        }
    ],
    inferenceConfig={"maxTokens": 2048, "temperature": 0},
    outputConfig={
        "textFormat": {
            "type": "json_schema",
            "structure": {
                "jsonSchema": {
                    "schema": json.dumps(invoice_schema),
                    "name": "invoice_extraction",
                    "description": "Extract structured invoice data"
                }
            }
        }
    }
)

invoice_data = json.loads(response["output"]["message"]["content"][0]["text"])

print("📊 Extracted Invoice Data:")
print("=" * 50)
print(json.dumps(invoice_data, indent=2))

print("\n📈 Summary:")
print(f"  Invoice: {invoice_data['invoice_number']}")
print(f"  Customer: {invoice_data['customer']['name']}")
print(f"  Line Items: {len(invoice_data['line_items'])}")
print(f"  Total: ${invoice_data['total_amount']:,.2f}")
print(f"  Status: {invoice_data['payment_status']}")

---

## 3. JSON Schema output with InvokeModel API

The InvokeModel API provides direct access to Claude's native request format. Use this when you need fine-grained control or are working with provider-specific features.

**Key difference:** The schema is specified in `output_config.format` rather than `outputConfig.textFormat`.

In [ ]:
# Schema for sentiment analysis with classification
sentiment_schema = {
    "type": "object",
    "properties": {
        "sentiment": {
            "type": "string",
            "enum": ["positive", "negative", "neutral", "mixed"],
            "description": "Overall sentiment of the text"
        },
        "confidence": {
            "type": "number",
            "description": "Confidence score between 0 and 1"
        },
        "key_phrases": {
            "type": "array",
            "items": {"type": "string"},
            "description": "Key phrases that influenced the sentiment"
        },
        "emotions": {
            "type": "array",
            "items": {
                "type": "string",
                "enum": ["joy", "sadness", "anger", "fear", "surprise", "trust", "anticipation", "disgust"]
            },
            "description": "Detected emotions"
        },
        "summary": {
            "type": "string",
            "description": "Brief summary of the sentiment analysis"
        }
    },
    "required": ["sentiment", "confidence", "key_phrases", "emotions", "summary"],
    "additionalProperties": False
}

# Sample review text
review_text = """
I've been using this product for three months now and I'm genuinely impressed. 
The build quality exceeded my expectations, and customer support was incredibly 
helpful when I had questions about setup. My only minor complaint is the 
documentation could be more detailed, but overall this has been a fantastic 
purchase. Highly recommended!
"""

# Build the InvokeModel request body
request_body = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1024,
    "temperature": 0,
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"Analyze the sentiment of this review:\n\n{review_text}"
                }
            ]
        }
    ],
    "output_config": {
        "format": {
            "type": "json_schema",
            "schema": sentiment_schema
        }
    }
}

# Make the request
response = bedrock_runtime.invoke_model(
    modelId=MODEL_ID,
    body=json.dumps(request_body)
)

# Parse the response
response_body = json.loads(response["body"].read())
sentiment_result = json.loads(response_body["content"][0]["text"])

print("📊 Sentiment Analysis Results:")
print("=" * 50)
print(f"Sentiment: {sentiment_result['sentiment'].upper()}")
print(f"Confidence: {sentiment_result['confidence']:.1%}")
print(f"\nEmotions detected: {', '.join(sentiment_result['emotions'])}")
print(f"\nKey phrases:")
for phrase in sentiment_result['key_phrases']:
    print(f"  • {phrase}")
print(f"\nSummary: {sentiment_result['summary']}")

---

## 4. Strict tool use with Converse API

Strict tool use validates that when the model calls a tool, the input parameters match your defined schema. This is essential for building reliable agentic systems.

**Key concept:** Set `strict: true` in your tool definition to enable schema validation on tool inputs.

### 4.1 Quick start: weather tool (matches blog example)

This simple example from the blog demonstrates the core concept of strict tool use.

In [ ]:
# Simple weather tool example (matches the blog)
response = bedrock_runtime.converse(
    modelId=MODEL_ID,
    messages=[
        {
            "role": "user",
            "content": [{"text": "What's the weather like in San Francisco?"}]
        }
    ],
    inferenceConfig={"maxTokens": 1024},
    toolConfig={
        "tools": [
            {
                "toolSpec": {
                    "name": "get_weather",
                    "description": "Get the current weather for a specified location",
                    "strict": True,  # Enable strict mode
                    "inputSchema": {
                        "json": {
                            "type": "object",
                            "properties": {
                                "location": {
                                    "type": "string",
                                    "description": "The city and state, e.g., San Francisco, CA"
                                },
                                "unit": {
                                    "type": "string",
                                    "enum": ["celsius", "fahrenheit"],
                                    "description": "Temperature unit"
                                }
                            },
                            "required": ["location", "unit"],
                            "additionalProperties": False
                        }
                    }
                }
            }
        ]
    }
)

# Tool inputs match the schema
print("🔧 Tool Call Requested:")
print("=" * 50)

for content_block in response["output"]["message"]["content"]:
    if "toolUse" in content_block:
        tool_input = content_block["toolUse"]["input"]
        print(f"Tool: {content_block['toolUse']['name']}")
        print(f"Input: {json.dumps(tool_input, indent=2)}")

print("\nWith strict: true, structured outputs validates that:")
print("  • The 'location' field is always a string")
print("  • The 'unit' field is always either 'celsius' or 'fahrenheit'")
print("  • No unexpected fields appear in the input")

### 4.2 Extended example: multi-tool travel booking

Building on the basics, this example shows multiple tools working together for a travel booking scenario.

In [ ]:
# Simulated tool implementations
def search_flights(origin, destination, departure_date, passengers, cabin_class):
    """Simulated flight search"""
    return {
        "flights": [
            {
                "flight_number": "AA100",
                "departure": "08:00",
                "arrival": "14:00+1",
                "price_per_person": 2500,
                "total_price": 2500 * passengers
            },
            {
                "flight_number": "JL001",
                "departure": "13:00",
                "arrival": "17:00+1",
                "price_per_person": 2800,
                "total_price": 2800 * passengers
            }
        ]
    }

def book_hotel(city, check_in, check_out, guests, room_type):
    """Simulated hotel booking"""
    return {
        "hotel": "Grand Tokyo Hotel",
        "room_type": room_type,
        "check_in": check_in,
        "check_out": check_out,
        "price_per_night": 350,
        "total_nights": 5,
        "total_price": 1750
    }

# Process tool calls
tool_results = []
messages = [
    {
        "role": "user",
        "content": [
            {
                "text": "I need to book a trip for 2 people from New York to Tokyo on March 15, 2024. We want business class flights and a deluxe hotel room for 5 nights."
            }
        ]
    }
]

# First call to get tool requests
response = bedrock_runtime.converse(
    modelId=MODEL_ID,
    messages=messages,
    inferenceConfig={"maxTokens": 1024},
    toolConfig={"tools": tools}
)

# Add assistant's response to messages
messages.append(response["output"]["message"])

# Process each tool call
for content_block in response["output"]["message"]["content"]:
    if "toolUse" in content_block:
        tool_call = content_block["toolUse"]
        tool_name = tool_call["name"]
        tool_input = tool_call["input"]
        
        print(f"⚙️ Executing tool: {tool_name}")
        print(f"   Input: {json.dumps(tool_input, indent=2)}")
        
        # Execute the appropriate tool
        if tool_name == "search_flights":
            result = search_flights(**tool_input)
        elif tool_name == "book_hotel":
            result = book_hotel(**tool_input)
        else:
            result = {"error": "Unknown tool"}
        
        print(f"   Result: {json.dumps(result, indent=2)}\n")
        
        tool_results.append({
            "toolResult": {
                "toolUseId": tool_call["toolUseId"],
                "content": [{"json": result}]
            }
        })

# Add tool results and get final response
messages.append({
    "role": "user",
    "content": tool_results
})

final_response = bedrock_runtime.converse(
    modelId=MODEL_ID,
    messages=messages,
    inferenceConfig={"maxTokens": 1024},
    toolConfig={"tools": tools}
)

print("\n" + "=" * 50)
print("📋 Final Response:")
print("=" * 50)
for content_block in final_response["output"]["message"]["content"]:
    if "text" in content_block:
        print(content_block["text"])

In [ ]:
# Define tools with strict mode enabled
tools = [
    {
        "toolSpec": {
            "name": "search_flights",
            "description": "Search for available flights between two cities",
            "strict": True,
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "origin": {
                            "type": "string",
                            "description": "Departure city or airport code"
                        },
                        "destination": {
                            "type": "string",
                            "description": "Arrival city or airport code"
                        },
                        "departure_date": {
                            "type": "string",
                            "format": "date",
                            "description": "Departure date in YYYY-MM-DD format"
                        },
                        "passengers": {
                            "type": "integer",
                            "description": "Number of passengers"
                        },
                        "cabin_class": {
                            "type": "string",
                            "enum": ["economy", "business", "first"],
                            "description": "Cabin class preference"
                        }
                    },
                    "required": ["origin", "destination", "departure_date", "passengers", "cabin_class"],
                    "additionalProperties": False
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "book_hotel",
            "description": "Book a hotel room in a specified city",
            "strict": True,
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "city": {
                            "type": "string",
                            "description": "City for hotel booking"
                        },
                        "check_in": {
                            "type": "string",
                            "format": "date",
                            "description": "Check-in date"
                        },
                        "check_out": {
                            "type": "string",
                            "format": "date",
                            "description": "Check-out date"
                        },
                        "guests": {
                            "type": "integer",
                            "description": "Number of guests"
                        },
                        "room_type": {
                            "type": "string",
                            "enum": ["standard", "deluxe", "suite"],
                            "description": "Room type preference"
                        }
                    },
                    "required": ["city", "check_in", "check_out", "guests", "room_type"],
                    "additionalProperties": False
                }
            }
        }
    }
]

# Make a request that triggers multiple tool calls
response = bedrock_runtime.converse(
    modelId=MODEL_ID,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "text": "I need to book a trip for 2 people from New York to Tokyo on March 15, 2024. We want business class flights and a deluxe hotel room for 5 nights."
                }
            ]
        }
    ],
    inferenceConfig={"maxTokens": 1024},
    toolConfig={"tools": tools}
)

print("🔧 Tool Calls Requested:")
print("=" * 50)

for content_block in response["output"]["message"]["content"]:
    if "toolUse" in content_block:
        tool_call = content_block["toolUse"]
        print(f"\n📌 Tool: {tool_call['name']}")
        print(f"   ID: {tool_call['toolUseId']}")
        print(f"   Input (schema-validated):")
        print(json.dumps(tool_call['input'], indent=6))

print("\n✅ All tool inputs match their defined schemas!")

### 4.3 Complete tool use workflow

Let's implement a complete workflow where we handle tool calls and return results.

---

## 5. Strict tool use with InvokeModel API

You can also use strict tool use with the InvokeModel API for direct model access.

In [ ]:
# Define tool for InvokeModel format
invoke_model_tools = [
    {
        "name": "get_weather",
        "description": "Get the current weather for a specified location",
        "strict": True,
        "input_schema": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g., San Francisco, CA"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "Temperature unit"
                }
            },
            "required": ["location", "unit"],
            "additionalProperties": False
        }
    }
]

request_body = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1024,
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What's the weather like in Seattle? I prefer Fahrenheit."
                }
            ]
        }
    ],
    "tools": invoke_model_tools
}

response = bedrock_runtime.invoke_model(
    modelId=MODEL_ID,
    body=json.dumps(request_body)
)

response_body = json.loads(response["body"].read())

print("🔧 Tool Call via InvokeModel:")
print("=" * 50)

for content_block in response_body["content"]:
    if content_block["type"] == "tool_use":
        print(f"Tool: {content_block['name']}")
        print(f"Input: {json.dumps(content_block['input'], indent=2)}")
        print("\n✅ Input matches the defined schema!")

---

## 6. Combining JSON outputs and strict tool use

For complex agentic workflows, you can use both features together. The model will call tools with validated inputs AND return a structured final response.

In [ ]:
# Schema for the final structured response
trip_summary_schema = {
    "type": "object",
    "properties": {
        "trip_summary": {
            "type": "string",
            "description": "Brief summary of the planned trip"
        },
        "total_estimated_cost": {
            "type": "number",
            "description": "Total estimated cost in USD"
        },
        "next_steps": {
            "type": "array",
            "items": {"type": "string"},
            "description": "Recommended next steps for the traveler"
        },
        "warnings": {
            "type": "array",
            "items": {"type": "string"},
            "description": "Any warnings or considerations"
        }
    },
    "required": ["trip_summary", "total_estimated_cost", "next_steps"],
    "additionalProperties": False
}

# Note: In a real implementation, you would:
# 1. First call with tools to get tool requests
# 2. Execute tools and return results
# 3. Final call with outputConfig to get structured summary

# For demonstration, here's how you'd structure the final call:
print("💡 Combined Usage Pattern:")
print("=" * 50)
print("""
When using both features together:

1. Tool calls with strict: true constrain parameters to match the schema
2. Final response with outputConfig constrains output to match the schema

Example flow:
  → User request
  → Model returns tool_use blocks (strict validation)
  → You execute tools
  → You return tool results
  → Model returns structured JSON (schema validation)
""")

# Simulated final response with structured output
demo_response = bedrock_runtime.converse(
    modelId=MODEL_ID,
    messages=[
        {
            "role": "user",
            "content": [{
                "text": """Based on this trip information, provide a summary:
                
                - Flight: NY to Tokyo, March 15, 2024, Business class, $5,000 for 2 passengers
                - Hotel: Grand Tokyo Hotel, Deluxe room, 5 nights, $1,750 total
                """
            }]
        }
    ],
    inferenceConfig={"maxTokens": 1024},
    outputConfig={
        "textFormat": {
            "type": "json_schema",
            "structure": {
                "jsonSchema": {
                    "schema": json.dumps(trip_summary_schema),
                    "name": "trip_summary",
                    "description": "Structured trip summary"
                }
            }
        }
    }
)

summary = json.loads(demo_response["output"]["message"]["content"][0]["text"])

print("\n📋 Structured Trip Summary:")
print("=" * 50)
print(json.dumps(summary, indent=2))

---

## 7. Error handling and edge cases

Even with structured outputs, you should handle certain edge cases gracefully.

In [ ]:
def safe_structured_request(model_id, messages, schema, schema_name, max_tokens=1024):
    """
    Make a structured output request with proper error handling.
    """
    try:
        response = bedrock_runtime.converse(
            modelId=model_id,
            messages=messages,
            inferenceConfig={"maxTokens": max_tokens},
            outputConfig={
                "textFormat": {
                    "type": "json_schema",
                    "structure": {
                        "jsonSchema": {
                            "schema": json.dumps(schema),
                            "name": schema_name,
                            "description": f"Schema for {schema_name}"
                        }
                    }
                }
            }
        )
        
        # Check stop reason
        stop_reason = response.get("stopReason", "unknown")
        
        if stop_reason == "max_tokens":
            print("⚠️ Warning: Response was truncated due to max_tokens limit")
            print("💡 Tip: Increase max_tokens and retry")
            return None, "max_tokens"
        
        if stop_reason == "refusal":
            print("⚠️ Warning: Model refused the request")
            return None, "refusal"
        
        # Parse the response
        result = json.loads(response["output"]["message"]["content"][0]["text"])
        return result, "success"
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        error_message = e.response['Error']['Message']
        print(f"❌ Error ({error_code}): {error_message}")
        
        if "schema" in error_message.lower():
            print("💡 Tip: Check that your schema uses only supported features")
            print("   - Set additionalProperties: false on all objects")
            print("   - Avoid recursive schemas")
            print("   - Check for unsupported constraints (min/max, etc.)")
        
        return None, error_code
    
    except json.JSONDecodeError as e:
        print(f"❌ JSON parsing error: {e}")
        return None, "json_error"

# Test the error handling
print("🧪 Testing Error Handling:")
print("=" * 50)

simple_schema = {
    "type": "object",
    "properties": {
        "greeting": {"type": "string"}
    },
    "required": ["greeting"],
    "additionalProperties": False
}

result, status = safe_structured_request(
    MODEL_ID,
    [{"role": "user", "content": [{"text": "Say hello!"}]}],
    simple_schema,
    "greeting"
)

if status == "success":
    print(f"✅ Success: {result}")

---

## 8. Common patterns and use cases

### 8.1 Classification with constrained categories

In [ ]:
# Schema for ticket classification
classification_schema = {
    "type": "object",
    "properties": {
        "category": {
            "type": "string",
            "enum": ["billing", "technical_support", "sales", "feedback", "other"],
            "description": "Primary category of the support ticket"
        },
        "priority": {
            "type": "string",
            "enum": ["low", "medium", "high", "urgent"],
            "description": "Priority level"
        },
        "sentiment": {
            "type": "string",
            "enum": ["positive", "neutral", "negative"],
            "description": "Customer sentiment"
        },
        "requires_escalation": {
            "type": "boolean",
            "description": "Whether the ticket needs immediate escalation"
        },
        "suggested_response_template": {
            "type": "string",
            "enum": ["acknowledgment", "troubleshooting", "refund_process", "feature_request", "general_inquiry"],
            "description": "Suggested response template to use"
        }
    },
    "required": ["category", "priority", "sentiment", "requires_escalation", "suggested_response_template"],
    "additionalProperties": False
}

tickets = [
    "I've been charged twice for my subscription this month and I want a refund immediately!",
    "Love your product! Just wondering if you have any plans to add dark mode?",
    "The API keeps returning 500 errors when I try to upload files larger than 10MB."
]

print("📋 Support Ticket Classification:")
print("=" * 50)

for i, ticket in enumerate(tickets, 1):
    response = bedrock_runtime.converse(
        modelId=MODEL_ID,
        messages=[{"role": "user", "content": [{"text": f"Classify this support ticket:\n\n{ticket}"}]}],
        inferenceConfig={"maxTokens": 256, "temperature": 0},
        outputConfig={
            "textFormat": {
                "type": "json_schema",
                "structure": {
                    "jsonSchema": {
                        "schema": json.dumps(classification_schema),
                        "name": "ticket_classification"
                    }
                }
            }
        }
    )
    
    classification = json.loads(response["output"]["message"]["content"][0]["text"])
    
    print(f"\n📌 Ticket {i}: \"{ticket[:50]}...\"")
    print(f"   Category: {classification['category']}")
    print(f"   Priority: {classification['priority']}")
    print(f"   Sentiment: {classification['sentiment']}")
    print(f"   Escalate: {'Yes' if classification['requires_escalation'] else 'No'}")
    print(f"   Template: {classification['suggested_response_template']}")

### 8.2 Multi-entity extraction

In [ ]:
# Schema for extracting multiple entities
entity_schema = {
    "type": "object",
    "properties": {
        "people": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "name": {"type": "string"},
                    "role": {"type": "string"},
                    "organization": {"type": "string"}
                },
                "required": ["name"],
                "additionalProperties": False
            }
        },
        "organizations": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "name": {"type": "string"},
                    "type": {
                        "type": "string",
                        "enum": ["company", "government", "nonprofit", "educational", "other"]
                    }
                },
                "required": ["name", "type"],
                "additionalProperties": False
            }
        },
        "dates": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "date": {"type": "string", "format": "date"},
                    "context": {"type": "string"}
                },
                "required": ["date", "context"],
                "additionalProperties": False
            }
        },
        "monetary_values": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "amount": {"type": "number"},
                    "currency": {"type": "string"},
                    "context": {"type": "string"}
                },
                "required": ["amount", "currency", "context"],
                "additionalProperties": False
            }
        }
    },
    "required": ["people", "organizations", "dates", "monetary_values"],
    "additionalProperties": False
}

news_article = """
TechCorp CEO Jane Smith announced today that the company has secured $50 million 
in Series C funding led by Venture Partners. The round also included participation 
from Innovation Capital. Smith stated that the funds will be used to expand into 
European markets by Q3 2024. The deal is expected to close on February 28, 2024. 
CFO Michael Chen noted that annual revenue had reached $25 million, representing 
150% year-over-year growth.
"""

response = bedrock_runtime.converse(
    modelId=MODEL_ID,
    messages=[{"role": "user", "content": [{"text": f"Extract all entities from this article:\n\n{news_article}"}]}],
    inferenceConfig={"maxTokens": 1024, "temperature": 0},
    outputConfig={
        "textFormat": {
            "type": "json_schema",
            "structure": {
                "jsonSchema": {
                    "schema": json.dumps(entity_schema),
                    "name": "entity_extraction"
                }
            }
        }
    }
)

entities = json.loads(response["output"]["message"]["content"][0]["text"])

print("📊 Extracted Entities:")
print("=" * 50)

print("\n👥 People:")
for person in entities["people"]:
    role = person.get("role", "N/A")
    org = person.get("organization", "N/A")
    print(f"   • {person['name']} - {role} at {org}")

print("\n🏢 Organizations:")
for org in entities["organizations"]:
    print(f"   • {org['name']} ({org['type']})")

print("\n📅 Dates:")
for date in entities["dates"]:
    print(f"   • {date['date']}: {date['context']}")

print("\n💰 Monetary Values:")
for value in entities["monetary_values"]:
    print(f"   • {value['currency']}{value['amount']:,.0f}: {value['context']}")

---

## 9. API comparison reference

Quick reference for using structured outputs across different APIs.

In [ ]:
print("""
📚 API Comparison Reference
================================================================================

┌─────────────────┬────────────────────────────────┬────────────────────────────┐
│ Feature         │ Converse API                   │ InvokeModel API            │
├─────────────────┼────────────────────────────────┼────────────────────────────┤
│ JSON Schema     │ outputConfig.textFormat        │ output_config.format       │
│ Location        │   .structure.jsonSchema        │   .schema                  │
├─────────────────┼────────────────────────────────┼────────────────────────────┤
│ Schema Format   │ JSON string in .schema field   │ JSON object directly       │
├─────────────────┼────────────────────────────────┼────────────────────────────┤
│ Strict Tool     │ toolSpec.strict: true          │ tools[].strict: true       │
├─────────────────┼────────────────────────────────┼────────────────────────────┤
│ Tool Schema     │ toolSpec.inputSchema.json      │ tools[].input_schema       │
├─────────────────┼────────────────────────────────┼────────────────────────────┤
│ Best For        │ Multi-turn conversations       │ Single-turn, native format │
└─────────────────┴────────────────────────────────┴────────────────────────────┘

🔑 Key Requirements (Both APIs):
  • additionalProperties: false on all objects
  • All required fields must be listed
  • Use supported schema features only
  • No recursive schemas
  
✅ Supported Schema Features:
  • Basic types: object, array, string, integer, number, boolean, null
  • enum (strings, numbers, bools, nulls only)
  • const, anyOf, allOf
  • $ref, $def, definitions (internal only)
  • String formats: date-time, time, date, email, uri, uuid, etc.
  • Array minItems: 0 or 1 only

❌ Not Supported:
  • Recursive schemas
  • External $ref
  • minimum, maximum, multipleOf
  • minLength, maxLength
  • additionalProperties: true or object
""")

---

## 10. Summary and key takeaways

### What we covered

1. **JSON Schema output format**: Control the model's response structure using `outputConfig.textFormat` (Converse) or `output_config.format` (InvokeModel)

2. **Strict tool use**: Validate tool parameters with `strict: true` for reliable agentic workflows

3. **Combined usage**: Use both features together for end-to-end schema validation

4. **Error handling**: Properly handle edge cases like refusals and token limits

5. **Common patterns**: Classification, entity extraction, data structuring

### Best practices

- ✅ Always set `additionalProperties: false` on all objects
- ✅ Use descriptive property names and descriptions
- ✅ Leverage `enum` for constrained values
- ✅ Check `stopReason` in responses
- ✅ Start with simple schemas and add complexity gradually
- ✅ Design for schema caching (24-hour cache for repeated schemas)

### Resources

- [Amazon Bedrock Documentation](https://docs.aws.amazon.com/bedrock/)
- [Converse API Reference](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html)
- [JSON Schema Specification](https://json-schema.org/)

---

## 🚀 Next steps

After exploring this notebook, you can:

- **Build data extraction pipelines** with schema-compliant outputs
- **Create intelligent classification systems** with constrained categories
- **Develop reliable agentic workflows** with validated tool parameters
- **Design API integrations** that trust model outputs without additional validation
- **Streamline production deployments** by reducing retry logic

---

Ready to build schema-compliant AI applications? Start experimenting with structured outputs on Amazon Bedrock today! 🎯